In [1]:
# 20231026回家作業說明
# 到環境部抓取aqx_p_07空氣品質監測站基本資料
# 參考網址 https://data.moenv.gov.tw/swagger/#/%E5%A4%A7%E6%B0%A3/get_aqx_p_07
# 建立password.py,提供api key,
# 建立連結抓取檔案（要確認csv檔或是json檔),
# 建立資料庫存取抓下來的資料
# 建立執行緒，進行每小時抓取一次資料

In [14]:
import password #匯入並執行password.py檔
import requests #匯入並執行requests
import sqlite3 #匯入sqlite3模組
import json #匯入json模組
def get_air_data():
    url=f'https://data.moenv.gov.tw/api/v2/aqx_p_319?api_key={password.apikey}'
    #print(url) #驗證url是否正確
    res=requests.get(url)
    data=res.json()['records']
    return data

In [3]:
def create_table_air(conn:sqlite3.Connection):
    #conn=sqlite3.Connection(database='air.db')
    cursor=conn.cursor()
    cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS AIR(
    "ID"    INTEGER,
    "監測日期"  TEXT NOT NULL,
    "監測站編號"    TEXT NOT NULL,
    "監測站名"  TEXT NOT NULL,
    "縣市行政區"    TEXT NOT NULL,
    "監測項目名稱"  TEXT NOT NULL,
    "監測數值"  TEXT NOT NULL,
    "監測單位"  TEXT NOT NULL,
    PRIMARY KEY("ID" AUTOINCREMENT)
    );
    '''
)
    conn.commit()

In [13]:
def insert_data_air(conn:sqlite3.Connection,values:list[any])->None:
    cursor=conn.cursor()
    sql='''
    REPLACE INTO AIR(監測日期,監測站編號,監測站名,縣市行政區,監測項目名稱,監測數值,監測單位)
    VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)
    conn.commit()

In [16]:
data=get_air_data()
conn=sqlite3.connect('air.db')
create_table_air(conn)
for item in data:
    #print(type(item))
    insert_data_air(conn,[item['monitordate'],item['siteid'],item['sitename'],item['county'],item['itemname'],item['concentration'],item['itemunit']])
conn.close()